In [1]:
%%capture
!git clone https://github.com/HackerBschor/SofteningQueryEvaluation
%cd SofteningQueryEvaluation

!pip3 install faiss-gpu-cu12
!pip3 install pgvector
!pip3 install kagglehub

In [2]:
from huggingface_hub import notebook_login
notebook_login()

# Evaluate Similarity Grouping


In [4]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from models import ModelMgr
from models.embedding.SentenceTransformer import SentenceTransformerEmbeddingModel
from models.semantic_validation import LLaMAValidationModel

from db.operators import Dummy, Select
from db.criteria import SoftEqual
from db.structure import Column, Constant
import kagglehub

from evaluation.util import calculate_metrics, calc_bleu

import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
# Models
# meta-llama/Llama-3.2-3B-Instruct
# meta-llama/Meta-Llama-3-8B-Instruct

stem = SentenceTransformerEmbeddingModel(ModelMgr())
lsv = LLaMAValidationModel(ModelMgr(), model_path = "meta-llama/Llama-3.2-3B-Instruct")
# lsv = DeepSeekValidationModel(m)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [16]:
ZERO_SHOT_SYSTEM_PROMPT = "You are a validator. Respond with \"no\" and \"yes\" only!"
ZERO_SHOT_PROMPTING_TEMPLATE = 'Does "{}" describes "{}"'

random_state = 420
max_number_categories = 100
max_number_per_categories = 1

target = "category_1"

In [7]:
path = kagglehub.dataset_download("lakritidis/product-classification-and-categorization")

df = pd.read_csv(f"{path}/pricerunner_aggregate.csv", header=None, index_col=0)
df.drop(columns=[2, 3, 5], inplace=True)
df.columns = ["product", "category_1", "category_2"]
df.head()

100%|██████████| 22.8M/22.8M [00:00<00:00, 182MB/s]

Extracting files...


,product,category_1,category_2
0,,,
1,apple iphone 8 plus 64gb silver,Apple iPhone 8 Plus 64GB,Mobile Phones
2,apple iphone 8 plus 64 gb spacegrau,Apple iPhone 8 Plus 64GB,Mobile Phones
3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,Apple iPhone 8 Plus 64GB,Mobile Phones
4,apple iphone 8 plus 64gb space grey,Apple iPhone 8 Plus 64GB,Mobile Phones
5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,Apple iPhone 8 Plus 64GB,Mobile Phones


In [25]:
categories = pd.Series(df[target].unique())
categories = list(categories.sample(min(max_number_categories, len(categories)), random_state=random_state))

print(len(categories)), print(", ".join(categories)[:100], "...")

100
Fujifilm X-T2 + 18-135mm OIS, Russell Hobbs RHM2064R Red, Samsung WW70K5410UX, Intel Xeon Silver 410 ...


(None, None)

In [26]:
df_reduced = pd.concat([df[df[target] == category].sample(max_number_per_categories, random_state=random_state) for category in categories])

gt = {(x["product"], x[target]) for key, x in df_reduced.iterrows()}
print(str(gt)[0:100], "...", len(gt))

{('nokia 1 dark blue', 'Nokia 1 Dual SIM'), ('ibm xeon l5530 2.4ghz 8mb l2 processor', 'IBM Intel Xe ... 100


In [27]:
overall_result = {}

def evaluate(method, threshold, system_prompt=ZERO_SHOT_SYSTEM_PROMPT, prompt_template=ZERO_SHOT_PROMPTING_TEMPLATE):
    pred = []
    runtimes = []
    for category in tqdm(categories):
        d = Dummy("products", ["product"], [(x[0], ) for x in gt])
        s = Select(d, SoftEqual(Column("product"),Constant(category), method=method, em=stem, sv=lsv, threshold=threshold, zfs_system_prompt=system_prompt, zfs_prompt_template = prompt_template))

        tic = time.time()
        result = s.open().fetch_all()
        toc = time.time()

        pred.extend([(x["product"], category) for x in result])
        runtimes.append(toc - tic)

    scores = calculate_metrics(gt, set(pred), np.mean(runtimes))

    print(method, threshold, scores["F1 Score"])

    return scores, pred

In [28]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("threshold", thresh)
    overall_result[("threshold", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 100/100 [02:08<00:00,  1.29s/it]


threshold 0.1 0.022862368541380886


100%|██████████| 100/100 [02:07<00:00,  1.27s/it]


threshold 0.2 0.031659737767828586


100%|██████████| 100/100 [02:07<00:00,  1.27s/it]


...

threshold 0.8 0.1272727272727273


100%|██████████| 100/100 [02:07<00:00,  1.28s/it]


threshold 0.9 0.0392156862745098


100%|██████████| 100/100 [02:07<00:00,  1.28s/it]

threshold 1.0 0


In [29]:
res, pred = evaluate("zero-few-shot", None)
overall_result[("zero-few-shot", None)] = res

100%|██████████| 100/100 [07:33<00:00,  4.53s/it]

zero-few-shot None 0.4


In [30]:
for thresh in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    res, _ = evaluate("both", thresh)
    overall_result[("both", thresh)] = res
    if res["Recall"] == 0.0:
        break

100%|██████████| 100/100 [08:40<00:00,  5.20s/it]


both 0.1 0.4


100%|██████████| 100/100 [06:48<00:00,  4.08s/it]


both 0.2 0.4


100%|██████████| 100/100 [04:38<00:00,  2.78s/it]


...

both 0.7 0.16513761467889906


100%|██████████| 100/100 [02:08<00:00,  1.29s/it]


both 0.8 0.019801980198019802


100%|██████████| 100/100 [02:08<00:00,  1.29s/it]

both 0.9 0


In [31]:
for key in tqdm(overall_result):
    scores_bleu = calc_bleu(gt, overall_result[key]["pred"])
    for score_bleu in scores_bleu:
        overall_result[key][score_bleu] = scores_bleu[score_bleu]

100%|██████████| 20/20 [08:39<00:00, 25.98s/it]


In [32]:
df_results = pd.DataFrame([{"method": k[0], "threshold": k[1]} | v for k,v in overall_result.items()])
df_results

,method,threshold,Precision,Recall,F1 Score,tp,fn,fp,runtime,pred,bleu1,bleu2,bleu3,bleu4
0,threshold,0.1,0.011563,1.00,0.022862,100,0,8548,1.284079,{(intel xeon silver 4108 1.8 ghz 8 core 16 thr...,1.000000,1.000000,1.000000,1.000000
1,threshold,0.2,0.016087,0.99,0.031660,99,1,6055,1.273690,{(intel xeon silver 4108 1.8 ghz 8 core 16 thr...,0.999130,0.998880,0.998776,0.998706
2,threshold,0.3,0.030415,0.99,0.059016,99,1,3156,1.273859,{(amica fk3216gbdf 297l frost free freestandin...,0.998696,0.998666,0.998634,0.998601
3,threshold,0.4,0.066202,0.95,0.123779,95,5,1340,1.275670,{(intel xeon silver 4108 1.8 ghz 8 core 16 thr...,0.988826,0.987912,0.987165,0.986690
4,threshold,0.5,0.128664,0.79,0.221289,79,21,535,1.274017,{(intel xeon silver 4108 1.8 ghz 8 core 16 thr...,0.907570,0.898635,0.891320,0.883406
5,threshold,0.6,0.222656,0.57,0.320225,57,43,199,1.276733,"{(aeg sce81816zf, Amica FK3216GBDF Black), (sa...",0.725531,0.682154,0.651081,0.632435
6,threshold,0.7,0.451613,0.28,0.345679,28,72,34,1.276413,{(cisco intel xeon e5504 2ghz 4mb l3 processor...,0.502126,0.428462,0.381359,0.365916
7,threshold,0.8,0.700000,0.07,0.127273,7,93,3,1.275589,"{(bosch kge49bi40, Bosch KGN34VB20G Black), (i...",0.248753,0.168662,0.130154,0.114702
8,threshold,0.9,1.000000,0.02,0.039216,2,98,0,1.275494,"{(sony xperia c4 dual sim smartphone, Sony Xpe...",0.114785,0.060205,0.043911,0.039168
9,threshold,1.0,0.000000,0.00,0.000000,0,100,0,1.274704,{},-1.000000,-1.000000,-1.000000,-1.000000


In [33]:
df_results.to_pickle("Products_fine_mpnetBaseV2_LLama8B.pkl")
df_results.drop(columns=["pred"]).to_csv("Products_fine_mpnetBaseV2_LLama8B.csv")